In [5]:
def showrst(df):
    print(df.to_markdown(tablefmt="grid"))

In [2]:
import pandas as pd
import nannyml as nml
from IPython.display import display

reference_df = nml.load_synthetic_multiclass_classification_dataset()[0]
analysis_df = nml.load_synthetic_multiclass_classification_dataset()[1]
analysis_target_df = nml.load_synthetic_multiclass_classification_dataset()[2]
analysis_df = analysis_df.merge(analysis_target_df, on='identifier')

display(reference_df.head(3))

,acq_channel,app_behavioral_score,requested_credit_limit,app_channel,credit_bureau_score,stated_income,is_customer,period,identifier,timestamp,y_pred_proba_prepaid_card,y_pred_proba_highstreet_card,y_pred_proba_upmarket_card,y_pred,y_true
0,Partner3,1.808232,350,web,309,15000,True,reference,60000,2020-05-02 02:01:30,0.97,0.03,0.00,prepaid_card,prepaid_card
1,Partner2,4.382568,500,mobile,418,23000,True,reference,60001,2020-05-02 02:03:33,0.87,0.13,0.00,prepaid_card,prepaid_card
2,Partner2,-0.787575,400,web,507,24000,False,reference,60002,2020-05-02 02:04:49,0.47,0.35,0.18,prepaid_card,upmarket_card


In [3]:
calc = nml.PerformanceCalculator(
    y_pred_proba={
        'prepaid_card': 'y_pred_proba_prepaid_card',
        'highstreet_card': 'y_pred_proba_highstreet_card',
        'upmarket_card': 'y_pred_proba_upmarket_card'
    },
    y_pred='y_pred',
    y_true='y_true',
    timestamp_column_name='timestamp',
    metrics=['f1', 'roc_auc'],
    chunk_size=6000)

calc.fit(reference_df)

In [4]:
results = calc.calculate(analysis_df)
display(results.data.head(3))

,key,start_index,end_index,start_date,end_date,period,targets_missing_rate,f1,f1_lower_threshold,f1_upper_threshold,f1_alert,roc_auc,roc_auc_lower_threshold,roc_auc_upper_threshold,roc_auc_alert
0,[0:5999],0,5999,2020-09-01 03:10:01,2020-09-13 16:15:10,None,0.0,0.751103,0.741254,0.764944,False,0.907595,0.900902,0.913516,False
1,[6000:11999],6000,11999,2020-09-13 16:15:32,2020-09-25 19:48:42,None,0.0,0.763045,0.741254,0.764944,False,0.910534,0.900902,0.913516,False
2,[12000:17999],12000,17999,2020-09-25 19:50:04,2020-10-08 02:53:47,None,0.0,0.758487,0.741254,0.764944,False,0.909414,0.900902,0.913516,False


In [6]:
showrst(results.data.head(3))

+----+---------------+---------------+-------------+---------------------+---------------------+----------+------------------------+----------+----------------------+----------------------+------------+-----------+---------------------------+---------------------------+-----------------+
|    | key           |   start_index |   end_index | start_date          | end_date            | period   |   targets_missing_rate |       f1 |   f1_lower_threshold |   f1_upper_threshold | f1_alert   |   roc_auc |   roc_auc_lower_threshold |   roc_auc_upper_threshold | roc_auc_alert   |
+====+===============+===============+=============+=====================+=====================+==========+========================+==========+======================+======================+============+===========+===========================+===========================+=================+
|  0 | [0:5999]      |             0 |        5999 | 2020-09-01 03:10:01 | 2020-09-13 16:15:10 |          |                      0 | 

In [7]:
engine='orca'
for metric in calc.metrics:
    figure = results.plot(kind='performance', plot_reference=True, metric=metric)
    #     fig.write_image(file=f"../_static/tutorial-perf-est-mc-guide-analysis-{metric}.svg", engine=engine)
    figure.show()